In [1]:
import tensorflow as tf

/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/s

In [2]:
export_dir = "/tmp/tfmodeldir/1"

In [3]:
input_placeholder = tf.placeholder(tf.int32, shape=[None, 1], name='input_placeholder')
output_placeholder = tf.identity(tf.add(input_placeholder, 2), name='output_placeholder')

In [4]:
sess = tf.Session()
tf.saved_model.simple_save(sess, export_dir, inputs={"key":input_placeholder}, outputs={"keys":output_placeholder})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfmodeldir/1/saved_model.pb


In [5]:
!tree "/tmp/tfmodeldir/1"

/tmp/tfmodeldir/1
├── saved_model.pb
└── variables

1 directory, 1 file


# Checking the saved model signature

In [6]:
!saved_model_cli show --dir /tmp/tfmodeldir/1 --all

/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/s

# Tensorflow Serving

In [ ]:
!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=simple_add --model_base_path=/tmp/tfmodeldir/

2019-10-23 01:32:25.761732: I tensorflow_serving/model_servers/server.cc:82] Building single TensorFlow model file config:  model_name: simple_add model_base_path: /tmp/tfmodeldir/
2019-10-23 01:32:25.761842: I tensorflow_serving/model_servers/server_core.cc:462] Adding/updating models.
2019-10-23 01:32:25.761853: I tensorflow_serving/model_servers/server_core.cc:561]  (Re-)adding model: simple_add
2019-10-23 01:32:25.862420: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: simple_add version: 1}
2019-10-23 01:32:25.862486: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: simple_add version: 1}
2019-10-23 01:32:25.862515: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: simple_add version: 1}
2019-10-23 01:32:25.862566: I external/org_tensorflow/tensorflow/contrib/session_bundle/bundle_shim.cc:363] Attempting to load native SavedModelBundle in bundle-shim from

# Client for inference

In [ ]:
import requests
import json

endpoint = "http://localhost:8501/v1/models/simple_add:predict"

instance = [[11,1],[22,1],[44,1],[55,1]]


headers = {"content-type":"application-json"}
data = json.dumps({"signature-name":"serving_default", "instances": instance})
response = requests.post(endpoint, data=data, headers=headers)
prediction = json.loads(response.text)['predictions']
prediction

In [13]:
# with tf.gfile.GFile("/tmp/tfmodeldir/1/saved_model.pb", "rb") as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())

     
# with tf.Graph().as_default() as graph:
#       tf.import_graph_def(graph_def,
#                           input_map=None,
#                           return_elements=None,
#                           name=""
#       )
        
        
frozen_graph="/tmp/tfmodeldir/1/saved_model.pb"

with tf.gfile.GFile(frozen_graph, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

IndentationError: unexpected indent (<ipython-input-13-0257f920ab98>, line 5)

In [ ]:
with tf.Graph().as_default() as graph:
      tf.import_graph_def(graph_def,
                          input_map=None,
                          return_elements=None,
                          name=""
      )

# Freezing model from the Keras 

In [1]:
import keras
import json
import os
import sys
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

Using TensorFlow backend.
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/PycharmProje

In [2]:
base_path = "/tmp/tfmodeldir/kerasmodel"

In [5]:
inception_model = InceptionV3(weights='imagenet', input_tensor=Input(shape=(299, 299, 3)))
inception_model.save(os.path.join(base_path,'inception.h5'))

In [6]:
#tf.keras.backend.set_learning_phase(0)

model = tf.keras.models.load_model(os.path.join(base_path, 'inception.h5'))
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 149, 149, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 149, 149, 32) 0           batch_normalization_189[0][0]    
_______________________________________________________________________________________

In [7]:
tf.keras.backend.set_learning_phase(0)
model = tf.keras.models.load_model(os.path.join(base_path, "inception.h5"))

export_dir = os.path.join('/tmp/tfmodeldir/kerasmodel', '1')


with tf.keras.backend.get_session() as sess:
    tf.saved_model.simple_save(sess, 
                              export_dir, 
                              inputs= {"keys":model.input},
                              outputs= {t.name: t for t in model.outputs})



# with tf.keras.backend.get_session() as sess:
#     tf.saved_model.simple_save(
#         sess,
#         export_dir,
#         inputs={'input_image': model.input},
#         outputs={t.name: t for t in model.outputs})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfmodeldir/kerasmodel/1/saved_model.pb
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]    

In [8]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
import requests

In [10]:
image = load_img('/home/jugs/JupyterNotebook/elephant.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
image = preprocess_input(image)

In [11]:
endpoint = "http://localhost:8501/v1/models/inception:predict"
headers = {"content-type":"application-json"}
instances = image.tolist()
data = json.dumps({"signature_name":"serving_default","instances": instances})
response = requests.post(endpoint, data=data, headers=headers)
prediction = json.loads(response.text)['predictions']
prediction[0].index(max(prediction[0]))

916